### 1. prep

In [ ]:
### import pyspark
import time
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import array, array_contains, col, concat_ws, explode, flatten, length, max, regexp_extract, regexp_replace, size, substring, when

In [ ]:
### create spark session
spark = SparkSession.builder \
        .appName("medistream-05") \
        .getOrCreate()

In [ ]:
### get start time
st = time.time()

In [ ]:
### read json data
path = path
data = spark.read.json(path)

### 2. preprocessing hospital dataframe

In [ ]:
hospital_data = data.select(explode("hospital").alias("h"))
hospital_df = hospital_data.select(
    col("h.id").alias("id"),
    col("h.name").alias("name"),
    col("h.category").alias("category"),
    col("h.category_code").alias("category_code"),
    col("h.category_code_list").alias("category_code_list"),
    col("h.category_count").alias("category_count"),
    col("h.description").alias("description"),
    col("h.road_address").alias("road_address"),
    col("h.road").alias("road"),
    col("h.rcode").alias("rcode"),
    col("h.virtual_phone").alias("virtual_phone"),
    col("h.phone").alias("phone"),
    col("h.payment_info").alias("payment_info"),
    col("h.conveniences").alias("conveniences"),
    col("h.review_setting.keyword").alias("review_keyword"),
    col("h.keywords").alias("keywords"),
    col("h.booking_business_id").alias("booking_business_id"),
    col("h.booking_display_name").alias("booking_display_name"),
    col("h.visitor_reviews_score").alias("visitor_reviews_score"),
    col("h.visitor_reviews_total").alias("visitor_reviews_total"),
    col("h.visitor_reviews_text_review_total").alias("visitor_reviews_text_review_total"),
    col("h.images").alias("images"),
    col("h.homepages.etc").alias("homepages_etc"),
    col("h.homepages.repr").alias("homepages_repr"),
    col("h.homepages.repr.url").alias("is_rep"), # isRep?
    col("h.booking_url").alias("booking_url"),
    col("h.talktalk_url").alias("talktalk_url"),
    col("h.coordinate.x").alias("lon"),
    col("h.coordinate.y").alias("lat"),
)
hospital_df = hospital_df.withColumn(
    "description",
    regexp_replace("description", "[\n\r*,]", "")
).withColumn(
    "road",
    regexp_replace("road", "[\n\r*,]", "")
).withColumn(
    "review_keyword",
    regexp_replace("review_keyword", "[\\\"]", "")
)
hospital_df = hospital_df.withColumn(
    "description_length",
    length("description")
).withColumn(
    "images_count", 
    size("images")
).withColumn(
    'photo_review_ratio',
    (col('visitor_reviews_total')-col('visitor_reviews_text_review_total'))/col('visitor_reviews_total')
).withColumn(
    'homepages_url', 
    flatten(array(array('homepages_repr.url'), 'homepages_etc.url'))
).withColumn(
    'homepages_type', 
    flatten(array(array('homepages_repr.type'), 'homepages_etc.type'))
).withColumn(
    'homepages_order', 
    when(
        col('homepages_repr.order').isNull(), 0
    ).otherwise(
        size(flatten(array(array('homepages_repr.order'), 'homepages_etc.order')))
    )
).withColumn(
    'is_smart_phone',
    col('phone').startswith('010')
).withColumn(
    'is_zero_pay',
    array_contains(col('payment_info'), '제로페이')
).withColumn(
    'is_dead_url',
    flatten(array(array('homepages_repr.isDeadUrl'), 'homepages_etc.isDeadUrl'))
).withColumn(
    'keywords_1',
    col('keywords')[0]
).withColumn(
    'keywords_2',
    col('keywords')[1]
).withColumn(
    'keywords_3',
    col('keywords')[2]
).withColumn(
    'keywords_4',
    col('keywords')[3]
).withColumn(
    'keywords_5',
    col('keywords')[4]
)
hospital_df = hospital_df.drop(
    "images", 
    "keywords", 
    "homepages_repr", 
    "homepages_etc"
).withColumn(
    "description",
    col("description").cast(StringType())
).withColumn(
    "road",
    col("road").cast(StringType())
).withColumn(
    "road_address",
    col("road_address").cast(StringType())   
).withColumn(
    "is_smart_phone", 
    col("is_smart_phone").cast(StringType())
).withColumn(
    "is_zero_pay", 
    col("is_zero_pay").cast(StringType())
)
arr_col_lst = [field.name for field in hospital_df.schema.fields if isinstance(field.dataType, ArrayType)]
for arr_col in arr_col_lst:
    hospital_df = hospital_df.withColumn(arr_col, concat_ws(",", arr_col))
print('hospital dataframe is ready!')

### 3. preprocessing root_query dataframe

In [ ]:
root_data = data.select(explode("root").alias("r"))
root_df = root_data.select(
    col("r.hospital.base.__ref").alias("root_id"),
    col("r.hospital.fsasReviews.total").alias("fsas_reviews_count"),
    col("r.hospital.kinQna.answerCount").alias("kin_qna_count")
)
root_df = root_df.withColumn(
    "root_id",
    regexp_extract("root_id", "HospitalBase:([\\w]+)", 1)
)

### 4. merge dataframes

In [ ]:
df = hospital_df.join(root_df, hospital_df.id == root_df.root_id, "left_outer")
df = df.drop("root_id")

### 5. save merged dataframe

In [ ]:
save_path = save_path
df.dropDuplicates()
df.write.mode('overwrite').parquet(save_path)

### 6. check task time

In [ ]:
ft = time.time()
print(f"pyspark task time: {ft - st}")

In [ ]:
columns = df.columns
for column in columns:
    max_length = df.select(max(length(col(column)))).collect()[0][0]
    print(f"Column: {column}, Max Length: {max_length}")

### 7. upload to redshift

In [ ]:
jdbc_url = jdbc_url
temp_dir = temp_dir
db_table = db_table

In [ ]:
df.write \
  .format("io.github.spark_redshift_community.spark.redshift") \
  .option("driver", "com.amazon.redshift.jdbc42.Driver") \
  .option("forward_spark_s3_credentials", True) \
  .option("url", jdbc_url) \
  .option("dbtable", db_table) \
  .option("tempdir", temp_dir) \
  .mode("overwrite") \
  .save()

In [ ]:
spark.stop()